In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of KalmanPaper.simple failed: Traceback (most recent call last):
  File "c:\Users\suzun\Dev\Paper\KalmanPaper\venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\suzun\Dev\Paper\KalmanPaper\venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
                      ^^^^^^^^
UnicodeDecodeError: 'cp932' codec can't decode byte 0x86 in position 490: illegal multibyte sequence
]


# Variational Approximation

> 変分近似

In [ ]:
#| default_exp VA

In [ ]:
#| hide
#| export
import jax.numpy as jnp
import jax.lax as lax
import jax.scipy.linalg as jsl
import jax
from jaxtyping import Array, Float, Int
from KalmanPaper import simple as sp
from typing import Tuple
from functools import partial

In [ ]:
#| export
def lam(
    x: Float[Array, ""] # $x$
) -> Float[Array, ""]: # $\lambda(x)$
  r"""$\!$*
  Lambda 関数
  $$\lambda(x)=\frac{1}{2x}\left[\sigma(x)-\frac{1}{2}\right]$$
  *$\!$"""
  return (sp.losi(x) - 0.5)/(2*x)

In [ ]:
#| hide
sp.reshow_doc(lam)

---

[source](https://github.com/SuzuSys/KalmanPaper/blob/main/KalmanPaper/simple.py#L64){target="_blank" style="float:right; font-size:smaller"}

### lam

>      lam (x:jaxtyping.Float[Array,''])

*$\!$*
Lambda 関数
$$\lambda(x)=\frac{1}{2x}\left[\sigma(x)-\frac{1}{2}\right]$$
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| x | Float[Array, ''] | $x$ |
| **Returns** | **Float[Array, '']** | **$\lambda(x)$** |

In [ ]:
#| export
@jax.jit
def Ptt(
    Ptm:  Float[Array, "N N"], # $\mathbf P_{t/t-1}$
    x:    Float[Array, "N"], # $\mathbf x_t$
    xi:   Float[Array, ""], # $\xi_t$
) -> Float[Array, "N N"]: # $\mathbf P_{t/t}$
  r"""$\!$*
  推定誤差共分散行列 $\mathbf P_{t/t}$
  $$\mathbf P_{t/t}=\mathbf P_{t/t-1}-\frac{2\lambda(\xi_t)}{1+2\lambda(\xi_t)\mathbf x_t^T\mathbf P_{t/t-1}\mathbf x_t}\left(\mathbf P_{t/t-1}\mathbf x_t\right)\left(\mathbf P_{t/t-1}\mathbf x_t\right)^T$$
  $$\mathbf P_{t/t}^{-1}=\mathbf P_{t/t-1}^{-1}+2\lambda(\xi_t)\mathbf x_t\mathbf x_t^T$$
  *$\!$"""
  dsigma = 2*lam(xi)
  Ptmx = Ptm @ x
  return Ptm - (dsigma / (1 + dsigma * (x @ Ptmx))) * jnp.outer(Ptmx, Ptmx)

In [ ]:
#| hide
sp.reshow_doc(Ptt)

---

[source](https://github.com/SuzuSys/KalmanPaper/blob/main/KalmanPaper/EKF.py#L17){target="_blank" style="float:right; font-size:smaller"}

### Ptt

>      Ptt (Ptm:jaxtyping.Float[Array,'NN'], x:jaxtyping.Float[Array,'N'],
>           xi:jaxtyping.Float[Array,''])

*$\!$*
推定誤差共分散行列 $\mathbf P_{t/t}$
$$\mathbf P_{t/t}=\mathbf P_{t/t-1}-\frac{2\lambda(\xi_t)}{1+2\lambda(\xi_t)\mathbf x_t^T\mathbf P_{t/t-1}\mathbf x_t}\left(\mathbf P_{t/t-1}\mathbf x_t\right)\left(\mathbf P_{t/t-1}\mathbf x_t\right)^T$$
$$\mathbf P_{t/t}^{-1}=\mathbf P_{t/t-1}^{-1}+2\lambda(\xi_t)\mathbf x_t\mathbf x_t^T$$
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| Ptm | Float[Array, 'N N'] | $\mathbf P_{t/t-1}$ |
| x | Float[Array, 'N'] | $\mathbf x_t$ |
| xi | Float[Array, ''] | $\xi_t$ |
| **Returns** | **Float[Array, 'N N']** | **$\mathbf P_{t/t}$** |

In [ ]:
#| export
@jax.jit
def wtt(
    Ptm: Float[Array, "N N"],  # $\mathbf P_{t/t-1}$
    Ptt_: Float[Array, "N N"], # $\mathbf P_{t/t}$
    w: Float[Array, "N"],      # $\hat{\mathbf w}_{t/t-1}$
    x: Float[Array, "N"],      # $\mathbf x_t$
    y: Float[Array, ""],      # $y_t$
) -> Float[Array, "N"]:        # $\hat{\mathbf w}_{t/t}$
  r"""$\!$*
  濾波推定値 $\hat{\mathbf w}_{t/t}$
  $$\hat{\mathbf w}_{t/t}=\mathbf P_{t/t}\left(\mathbf P_{t/t-1}^{-1}\hat{\mathbf w}_{t/t-1}+(y_t-1/2)\mathbf x_t\right)$$
  *$\!$"""
  return Ptt_ @ (jsl.cho_solve(jsl.cho_factor(Ptm), w) + (y - 1/2) * x)

In [ ]:
#| hide
sp.reshow_doc(wtt)

---

[source](https://github.com/SuzuSys/KalmanPaper/blob/main/KalmanPaper/EKF.py#L33){target="_blank" style="float:right; font-size:smaller"}

### wtt

>      wtt (Ptm:jaxtyping.Float[Array,'NN'], Ptt_:jaxtyping.Float[Array,'NN'],
>           w:jaxtyping.Float[Array,'N'], x:jaxtyping.Float[Array,'N'],
>           y:jaxtyping.Float[Array,''])

*$\!$*
濾波推定値 $\hat{\mathbf w}_{t/t}$
$$\hat{\mathbf w}_{t/t}=\mathbf P_{t/t}\left(\mathbf P_{t/t-1}^{-1}\hat{\mathbf w}_{t/t-1}+(y_t-1/2)\mathbf x_t\right)$$
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| Ptm | Float[Array, 'N N'] | $\mathbf P_{t/t-1}$ |
| Ptt_ | Float[Array, 'N N'] | $\mathbf P_{t/t}$ |
| w | Float[Array, 'N'] | $\hat{\mathbf w}_{t/t-1}$ |
| x | Float[Array, 'N'] | $\mathbf x_t$ |
| y | Float[Array, ''] | $y_t$ |
| **Returns** | **Float[Array, 'N']** | **$\hat{\mathbf w}_{t/t}$** |

### `xit`: $(\mathbf P_{t/t-1},\hat{\mathbf w}_{t/t-1})$ を使う変分パラメータ $\xi_t$

$$\xi_t=\sqrt{\mathbf x_t^T\left(\mathbf P_{t/t-1}+\hat{\mathbf w}_{t/t-1}\hat{\mathbf w}_{t/t-1}^T\right)\mathbf x_t}$$

In [ ]:
#| export
@jax.jit
def xit(
    Cov: Float[Array, "N N"], # $\boldsymbol\Sigma$
    w: Float[Array, "N"],   # $\hat{\mathbf w}$
    x: Float[Array, "N"],   # $\mathbf x_t$
) -> Float[Array, ""]: # $\xi_t$
  r"""$\!$*
  変分パラメータ $\xi_t$
  $$\xi_t=\sqrt{\mathbf x_t^T\mathbb E[\mathbf w\mathbf w^T]\mathbf x_t}=\sqrt{\mathbf x_t^T\left(\boldsymbol \Sigma+\hat{\mathbf w}\hat{\mathbf w}^T\right)\mathbf x_t}$$
  *$\!$"""
  return jnp.sqrt(x @ (Cov + jnp.outer(w,w)) @ x)

In [ ]:
#| hide
sp.reshow_doc(xit)

---

[source](https://github.com/SuzuSys/KalmanPaper/blob/main/KalmanPaper/VA.py#L42){target="_blank" style="float:right; font-size:smaller"}

### xit

>      xit (Cov:jaxtyping.Float[Array,'NN'], w:jaxtyping.Float[Array,'N'],
>           x:jaxtyping.Float[Array,'N'])

*$\!$*
変分パラメータ $\xi_t$
$$\xi_t=\sqrt{\mathbf x_t^T\mathbb E[\mathbf w\mathbf w^T]\mathbf x_t}=\sqrt{\mathbf x_t^T\left(\boldsymbol \Sigma+\hat{\mathbf w}\hat{\mathbf w}^T\right)\mathbf x_t}$$
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| Cov | Float[Array, 'N N'] | $\boldsymbol\Sigma$ |
| w | Float[Array, 'N'] | $\hat{\mathbf w}$ |
| x | Float[Array, 'N'] | $\mathbf x_t$ |
| **Returns** | **Float[Array, '']** | **$\xi_t$** |

### `VApre`: `xipre` を使う Variational Approximation

$$N,T,\{ \mathbf x_t \}_{t=0,\ldots,T-1}, \{ y_t \}_{t=0,\ldots,T-1}, \boldsymbol\Gamma, \hat{\mathbf w}_{0/-1}, \mathbf P_{0/-1} $$
$$\to\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\{\xi_t\}_{t=0,\ldots,T-1}$$

In [ ]:
#| export
@partial(jax.jit, static_argnames=['N', 'T'])
def VApre(
    N: int, # $N$
    T: int, # $T$
    x: Float[Array, "{T} {N}"], # $\{ \mathbf x_t \}_{t=0,\ldots,T-1}$
    y: Float[Array, "{T} {N}"], # $\{ y_t \}_{t=0,\ldots,T-1}$
    G: Float[Array, "{N} {N}"], # $\boldsymbol\Gamma$
    w0: Float[Array, "{N}"], # $\hat{\mathbf w}_{0/-1}$
    P0: Float[Array, "{N} {N}"], # $\mathbf P_{0/-1}$
) -> Tuple[Float[Array, "{T} {N}"], Float[Array, "{T} {N} {N}"], Float[Array, "{T}"]]: # $\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\{\xi_t\}_{t=0,\ldots,T-1}$
    r"""$\!$*
    一段予測推定値 $\hat{\mathbf w}_{t/t-1}$ を使う変分近似法
    $$\xi_t=\sqrt{\mathbf x_t^T\left(\mathbf P_{t/t-1}+\hat{\mathbf w}_{t/t-1}\hat{\mathbf w}_{t/t-1}^T\right)\mathbf x_t}$$
    *$\!$"""
    def step(carry, inputs):
        Ptm, wtm = carry
        xt, yt = inputs
        xit_ = xit(Ptm, wtm, xt)
        Ptt_ = Ptt(Ptm, xt, xit_)
        wtt_ = wtt(Ptm, Ptt_, wtm, xt, yt)
        return (Ptt_ + G, wtt_), (wtt_, Ptt_, xit_)
    
    _, (W, P, Xi) = lax.scan(
        step,
        (P0, w0),
        (x, y),
        length=T
    )
    return W, P, Xi

In [ ]:
#| hide
sp.reshow_doc(VApre)

---

[source](https://github.com/SuzuSys/KalmanPaper/blob/main/KalmanPaper/VA.py#L51){target="_blank" style="float:right; font-size:smaller"}

### VApre

>      VApre (N:int, T:int, x:jaxtyping.Float[Array,'{T}{N}'],
>             y:jaxtyping.Float[Array,'{T}{N}'],
>             G:jaxtyping.Float[Array,'{N}{N}'],
>             w0:jaxtyping.Float[Array,'{N}'],
>             P0:jaxtyping.Float[Array,'{N}{N}'])

*$\!$*
一段予測推定値 $\hat{\mathbf w}_{t/t-1}$ を使う変分近似法
$$\xi_t=\sqrt{\mathbf x_t^T\left(\mathbf P_{t/t-1}+\hat{\mathbf w}_{t/t-1}\hat{\mathbf w}_{t/t-1}^T\right)\mathbf x_t}$$
*$\!$*

|$\!$| **Type** | **Details** |
| -- | -------- | ----------- |
| N | int | $N$ |
| T | int | $T$ |
| x | Float[Array, '{T} {N}'] | $\{ \mathbf x_t \}_{t=0,\ldots,T-1}$ |
| y | Float[Array, '{T} {N}'] | $\{ y_t \}_{t=0,\ldots,T-1}$ |
| G | Float[Array, '{N} {N}'] | $\boldsymbol\Gamma$ |
| w0 | Float[Array, '{N}'] | $\hat{\mathbf w}_{0/-1}$ |
| P0 | Float[Array, '{N} {N}'] | $\mathbf P_{0/-1}$ |
| **Returns** | **Tuple** | **$\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\{\xi_t\}_{t=0,\ldots,T-1}$** |

In [ ]:
#| export
@partial(jax.jit, static_argnames=['N', 'T'])
def VAEM(
    N: int, # $N$
    T: int, # $T$
    x: Float[Array, "{T} {N}"], # $\{ \mathbf x_t \}_{t=0,\ldots,T-1}$
    y: Float[Array, "{T} {N}"], # $\{ y_t \}_{t=0,\ldots,T-1}$
    G: Float[Array, "{N} {N}"], # $\boldsymbol\Gamma$
    w0: Float[Array, "{N}"], # $\hat{\mathbf w}_{0/-1}$
    P0: Float[Array, "{N} {N}"], # $\mathbf P_{0/-1}$
    epsilon: Float, # $\epsilon\ge |\xi\\^{\text{new}}_t-\xi\\^{\text{old}}_t|$
    max_iter: int = 100, # $繰り返し回数の上限$
) -> Tuple[Float[Array, "{T} {N}"], Float[Array, "{T} {N} {N}"], Float[Array, "{T}"]]: # $\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\{\xi_t\}_{t=0,\ldots,T-1}$
    r"""$\!$*
    濾波推定値 $\hat{\mathbf w}_{t/t}$ を使う変分近似法。EMアルゴリズムを使う。
    $$\xi_t=\sqrt{\mathbf x_t^T\left(\mathbf P_{t/t}+\hat{\mathbf w}_{t/t}\hat{\mathbf w}_{t/t}^T\right)\mathbf x_t}$$
    *$\!$"""
    def inner_iter(state):
        xit_pr, xit_af, Ptt_, wtt_, Ptm, wtm, xt, yt, i = state
        xit_pr = xit_af
        xit_af = xit(Ptt_, wtt_, xt)
        Ptt_new = Ptt(Ptm, xt, xit_af)
        wtt_new = wtt(Ptm, Ptt_new, wtm, xt, yt)
        return (xit_pr, xit_af, Ptt_new, wtt_new, Ptm, wtm, xt, yt, i+1)

    def cond_fun(state):
        xit_pr, xit_af, *_, i = state
        return jnp.logical_and(
            jnp.abs(xit_pr - xit_af) >= epsilon,
            i < max_iter
        )

    def step(carry, inputs):
        Ptm, wtm = carry
        xt, yt = inputs
        xit_pr = xit(Ptm, wtm, xt)
        Ptt_ = Ptt(Ptm, xt, xit_pr)
        wtt_ = wtt(Ptm, Ptt_, wtm, xt, yt)
        xit_af = xit(Ptt_, wtt_, xt)

        init_state = (xit_pr, xit_af, Ptt_, wtt_, Ptm, wtm, xt, yt, 0)
        _, xit_final, Ptt_final, wtt_final, *_ = lax.while_loop(cond_fun, inner_iter, init_state)

        return (Ptt_final + G, wtt_final), (wtt_final, Ptt_final, xit_final)

    _, (W, P, Xi) = lax.scan(step, (P0, w0), (x, y), length=T)
    return W, P, Xi

In [ ]:
#| hide
sp.reshow_doc(VAEM)

---

[source](https://github.com/SuzuSys/KalmanPaper/blob/main/KalmanPaper/VA.py#L78){target="_blank" style="float:right; font-size:smaller"}

### VAEM

>      VAEM (N:int, T:int, x:jaxtyping.Float[Array,'{T}{N}'],
>            y:jaxtyping.Float[Array,'{T}{N}'],
>            G:jaxtyping.Float[Array,'{N}{N}'], w0:jaxtyping.Float[Array,'{N}'],
>            P0:jaxtyping.Float[Array,'{N}{N}'], epsilon:jaxtyping.Float,
>            max_iter:int=100)

*$\!$*
濾波推定値 $\hat{\mathbf w}_{t/t}$ を使う変分近似法。EMアルゴリズムを使う。
$$\xi_t=\sqrt{\mathbf x_t^T\left(\mathbf P_{t/t}+\hat{\mathbf w}_{t/t}\hat{\mathbf w}_{t/t}^T\right)\mathbf x_t}$$
*$\!$*

|$\!$| **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| N | int |$\!$| $N$ |
| T | int |$\!$| $T$ |
| x | Float[Array, '{T} {N}'] |$\!$| $\{ \mathbf x_t \}_{t=0,\ldots,T-1}$ |
| y | Float[Array, '{T} {N}'] |$\!$| $\{ y_t \}_{t=0,\ldots,T-1}$ |
| G | Float[Array, '{N} {N}'] |$\!$| $\boldsymbol\Gamma$ |
| w0 | Float[Array, '{N}'] |$\!$| $\hat{\mathbf w}_{0/-1}$ |
| P0 | Float[Array, '{N} {N}'] |$\!$| $\mathbf P_{0/-1}$ |
| epsilon | Float |$\!$| $\epsilon\ge \|\xi\\^{\text{new}}_t-\xi\\^{\text{old}}_t\|$ |
| max_iter | int | 100 | $繰り返し回数の上限$ |
| **Returns** | **Tuple** |$\!$| **$\{\hat{\mathbf w}_{t/t}\}_{t=0,\ldots,T-1},\{\mathbf P_{t/t}\}_{t=0,\ldots,T-1},\{\xi_t\}_{t=0,\ldots,T-1}$** |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()